<a href="https://colab.research.google.com/github/GinkGoPi/researches/blob/main/token-stake/simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simeple about token staking and reward

> 模拟简单模型的staking和reward的逻辑和计算

In [1]:
import calendar
import time

In [2]:
calendar.timegm(time.gmtime())

1675235779

In [22]:
class SimepleStakingRewards(object):
    
    def __init__(self) -> None:
        self.stakingToken = 'stakingToken'
        self.rewardsToken = 'rewardsToken'

        self.periodFinish = 0
        self.lastUpdateTime = 0
        self.duration = 1 * 60*60*24 # seconds

        self.rewardRate = 0
        self.rewardPerTokenStored = 0

        self.userRewardPerTokenPaid = {'address_0': 0, 'alice': 0, 'bob': 0, 'kite': 0}
        self.rewards = {'address_0': 0, 'alice': 0, 'bob': 0, 'kite': 0}

        self.totalSupply = 0
        self.balanceOf = {'address_0': 0, 'alice': 0, 'bob': 0, 'kite': 0}
    
    def stake(self, address: str, amount: int):
        self.updateReward(address)
        self.balanceOf[address] += amount
        self.totalSupply += amount
    
    def withdraw(self, to: str, amount: int):
        self.updateReward(to)
        self.balanceOf[to] -= amount
        self.totalSupply -= amount
        return amount
      
    def updateReward(self, _account: str):
        self.rewardPerTokenStored = self.rewardPerToken()
        self.lastUpdateTime = self.lastTimeRewardApplicable()

        if _account != "address_0":
            self.rewards[_account] = self.earned(_account)
            self.userRewardPerTokenPaid[_account] = self.rewardPerTokenStored
    
    def rewardPerToken(self):
        if self.totalSupply == 0:
            return self.rewardPerTokenStored
        else:
            return self.rewardPerTokenStored + (self.rewardRate * (self.lastTimeRewardApplicable() - self.lastUpdateTime) * 1e18) / self.totalSupply

    def earned(self, _account: str):
        return ((self.balanceOf[_account] *
                (self.rewardPerToken() - self.userRewardPerTokenPaid[_account])) / 1e18) + self.rewards[_account]

    def getReward(self, to: str):
        """Gain reward token"""
        self.updateReward(to)
        reward = self.rewards[to]
        if reward > 0:
            self.rewards[to] = 0
            # rewardsToken.transfer(to, reward)
        
        return reward

    def notifyRewardAmount(self, _amount: int):
        self.updateReward("address_0")
        if self.blockTimestamp() >= self.periodFinish:
            self.rewardRate = _amount / self.duration
        else:
            remainingRewards = (self.periodFinish - self.blockTimestamp()) * self.rewardRate
            self.rewardRate = (_amount + remainingRewards) / self.duration
        

        assert(self.rewardRate > 0, "reward rate = 0")

        self.lastUpdateTime = self.blockTimestamp()
        self.periodFinish = self.blockTimestamp() + self.duration
        
    
    def lastTimeRewardApplicable(self):
        return self.periodFinish if self.periodFinish <= self.blockTimestamp() else self.blockTimestamp()

    def blockTimestamp(self):
        """Akin to `block.timestamp`"""
        return calendar.timegm(time.gmtime())



<>:68: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:68: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-22-8bff1e5d3881>:68: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(self.rewardRate > 0, "reward rate = 0")


In [23]:
# init
pool = SimepleStakingRewards()

print("init stake total amount", pool.totalSupply)
print("init stake rewardRate", pool.rewardRate)

init stake total amount 0
init stake rewardRate 0


In [24]:
# add new rewards
rewards = 10e18

pool.notifyRewardAmount(rewards)

print("current total amount", pool.totalSupply)
print("current rewardRate", pool.rewardRate)
print("current periodFinish", pool.periodFinish)
print("current lastUpdateTime", pool.lastUpdateTime)
print("current rewardPerTokenStored", pool.rewardPerTokenStored)

current total amount 0
current rewardRate 115740740740740.73
current periodFinish 1675324631
current lastUpdateTime 1675238231
current rewardPerTokenStored 0


In [25]:
# Alice staking

alice = "alice"
stakingAmount = 100e18

pool.stake(alice, stakingAmount)

print("current total amount", pool.totalSupply)
print("current rewardRate", pool.rewardRate)
print("current periodFinish", pool.periodFinish)
print("current lastUpdateTime", pool.lastUpdateTime)

print("current rewardPerTokenStored", pool.rewardPerTokenStored)

print("alice staked", pool.balanceOf[alice])
print("alice rewards", pool.rewards[alice])
print("alice userRewardPerTokenPaid", pool.userRewardPerTokenPaid[alice])


current total amount 1e+20
current rewardRate 115740740740740.73
current periodFinish 1675324631
current lastUpdateTime 1675238234
current rewardPerTokenStored 0
alice staked 1e+20
alice rewards 0.0
alice userRewardPerTokenPaid 0


In [31]:
print("T1 query alice current earned", pool.earned(alice))

T1 query alice current earned 1.3927469135802468e+16


In [32]:
print("T2 query alice current earned", pool.earned(alice))

T2 query alice current earned 1.404320987654321e+16


In [29]:
# Bob staking

bob = "bob"
stakingAmount = 200e18

pool.stake(bob, stakingAmount)

print("current total amount", pool.totalSupply)
print("current rewardRate", pool.rewardRate)
print("current periodFinish", pool.periodFinish)
print("current lastUpdateTime", pool.lastUpdateTime)

print("current rewardPerTokenStored", pool.rewardPerTokenStored)

print("bob staked", pool.balanceOf[bob])
print("bob rewards", pool.rewards[bob])
print("bob userRewardPerTokenPaid", pool.userRewardPerTokenPaid[bob])

current total amount 3e+20
current rewardRate 115740740740740.73
current periodFinish 1675324631
current lastUpdateTime 1675238344
current rewardPerTokenStored 127314814814814.81
bob staked 2e+20
bob rewards 0.0
bob userRewardPerTokenPaid 127314814814814.81


In [33]:
print("T1 query bob current earned", pool.earned(bob))


T1 query bob current earned 2932098765432100.0


In [34]:
print("T2 query bob current earned", pool.earned(bob))


T2 query bob current earned 3163580246913581.0


In [35]:
# Bob staking again

bob = "bob"
stakingAmount = 100e18

pool.stake(bob, stakingAmount)

print("current total amount", pool.totalSupply)
print("current rewardRate", pool.rewardRate)
print("current periodFinish", pool.periodFinish)
print("current lastUpdateTime", pool.lastUpdateTime)

print("current rewardPerTokenStored", pool.rewardPerTokenStored)

print("bob staked", pool.balanceOf[bob])
print("bob rewards", pool.rewards[bob])
print("bob userRewardPerTokenPaid", pool.userRewardPerTokenPaid[bob])

current total amount 4e+20
current rewardRate 115740740740740.73
current periodFinish 1675324631
current lastUpdateTime 1675238449
current rewardPerTokenStored 167824074074074.06
bob staked 3e+20
bob rewards 8101851851851850.0
bob userRewardPerTokenPaid 167824074074074.06


In [36]:
print("T3 query bob current earned", pool.earned(bob))


T3 query bob current earned 9577546296296290.0


In [37]:
print("T4 query bob current earned", pool.earned(bob))


T4 query bob current earned 9837962962962960.0


In [39]:
# alice withdraw and getReward

unstakingAmount = 50e18
unstakedAmount = pool.withdraw(alice, unstakingAmount)

print('alice unstaked amount', unstakedAmount)

print("current total amount", pool.totalSupply)
print("current rewardRate", pool.rewardRate)
print("current periodFinish", pool.periodFinish)
print("current lastUpdateTime", pool.lastUpdateTime)

print("current rewardPerTokenStored", pool.rewardPerTokenStored)

alice unstaked amount 5e+19
current total amount 3.5e+20
current rewardRate 115740740740740.73
current periodFinish 1675324631
current lastUpdateTime 1675238780
current rewardPerTokenStored 263599537037037.03


In [40]:
print("alice staked", pool.balanceOf[alice])
print("alice rewards", pool.rewards[alice])
print("alice userRewardPerTokenPaid", pool.userRewardPerTokenPaid[alice])

print("Tn query alice current earned", pool.earned(alice))

alice staked 5e+19
alice rewards 2.6359953703703704e+16
alice userRewardPerTokenPaid 263599537037037.03
Tn query alice current earned 2.688905423280423e+16


In [41]:
# alice getReward

aliceRewards = pool.getReward(alice)

print('alice getReward amount', aliceRewards)


alice getReward amount 2.8410218253968256e+16


In [42]:
print("current total amount", pool.totalSupply)
print("current rewardRate", pool.rewardRate)
print("current periodFinish", pool.periodFinish)
print("current lastUpdateTime", pool.lastUpdateTime)

print("alice staked", pool.balanceOf[alice])
print("alice rewards", pool.rewards[alice])
print("alice userRewardPerTokenPaid", pool.userRewardPerTokenPaid[alice])

print("Tn query alice current earned", pool.earned(alice))

current total amount 3.5e+20
current rewardRate 115740740740740.73
current periodFinish 1675324631
current lastUpdateTime 1675238904
alice staked 5e+19
alice rewards 0
alice userRewardPerTokenPaid 304604828042328.06
Tn query alice current earned 82671957671956.25
